Perguntas

1- O que a classe social (representada por Pclass) diz sobre as prioridades e o valor de uma vida humana naquela época?

2- A idade e o gênero determinavam o direito à vida? Qual a relação entre a lógica dos dados e a ética do "mulheres e crianças primeiro"?

3- O que os dados faltantes (missing values) nos dizem sobre o que não sabemos? Isso pode representar a incerteza e o caos de um evento como o naufrágio.

4- O conhecimento sobre o passado (os dados dos passageiros) pode prever o futuro (a sobrevivência ou não de um novo passageiro)?

5- Após a criação do modelo ML, ele é bom em prever quem sobreviveu, ou ele é bom em prever quem não sobreviveu? Ou ele é bom nos dois? O que os erros do modelo ensina sobre a incerteza da previsão?

6- Existem outras formas de "aprender" os padrões dos dados? Um modelo mais complexo (e menos intuitivo) pode oferecer uma previsão mais precisa?

7- Qual a verdadeira "precisão" do modelo? Ele é confiável em prever a vida e a morte?

8- A previsão é uma coincidência do destino, ou é uma verdade estatística consistente?

9- Como a união de informações pode criar um novo conhecimento que era invisível antes?

10- O novo conhecimento, através das mudanças na questão anterior, melhora a capacidade de prever o futuro?



In [ ]:
import pandas as pd
import numpy as np

# path do dataset a ser analisado
df = pd.read_csv('../data/Titanic-Dataset.csv')

# Exibir as cinco primeiras linhas
df.head()

- Entendimento e Preparação dos dados

In [ ]:
# criacao de coluna "Title"
import re

# Cria uma nova coluna 'Title' extraindo o título do nome
df['Title'] = df['Name'].apply(
    lambda x: re.search('([A-Za-z]+)\.', x).group(1)
)
df.head()

In [ ]:
# Verificar frequencia dos titulos
title_counts = df['Title'].value_counts()
print(title_counts)

In [ ]:
# Agrupar os títulos raros em uma única categoria "Rare" para simplificar as analises

titulos_raros = [
    'Dr', 'Rev', 'Mlle', 'Major', 'Col', 'Countess', 'Capt',
    'Ms', 'Sir', 'Lady', 'Mme', 'Don', 'Jonkheer'
]

# Criar dicionário para o mapeamento
mapeamento_titulos = {titulo: 'Rare' for titulo in titulos_raros}
mapeamento_titulos['Mlle'] = 'Miss'
mapeamento_titulos['Ms'] = 'Miss'
mapeamento_titulos['Mme'] = 'Mrs'

# Aplica o mapeamento
df['Title'] = df['Title'].replace(mapeamento_titulos)

print(df['Title'].value_counts())

In [ ]:
# Valores ausentes por coluna
valores_ausentes = df.isnull().sum()

# Calcular a porcentagem de valores ausentes
porcentagem_ausentes = (valores_ausentes / len(df)) * 100

# Criar DataFrame para visualizar a contagem e a porcentagem
df_valores_ausentes = pd.DataFrame({
    'Contagem': valores_ausentes,
    'Porcentagem': porcentagem_ausentes
})

# Filtrar apenas as colunas com valores ausentes e ordenamos
print(df_valores_ausentes[df_valores_ausentes['Contagem'] > 0].sort_values(by='Porcentagem', ascending=False))

- Analise dos dados acima: 

- Cabin: : Quase 80% dos dados estão faltando, inferir sobre estas informações é inviavel. Então melhor descartar esta coluna.
- Age:: A idade é uma variavel crucial, 20% faltante é uma quantidade consideravel.
- Embarked: Menos de 1% de dados ausentes, dois valores estão faltando. Posso preencher com o valor mais frequente tranquilamente.

In [ ]:
# Preencher idades faltantes

df['Age'] = df['Age'].fillna(
    df.groupby('Title')['Age'].transform('median')
)
print(df['Age'].isnull().sum())

In [ ]:
# Preencher o Porto de Embarque

porto_mais_comum = df['Embarked'].mode()[0]

df['Embarked'] = df['Embarked'].fillna(porto_mais_comum)

print(df['Embarked'].isnull().sum())

 1- Respondendo a primeira pergunta:

 O que a classe social (representada por Pclass) diz sobre as prioridades e o valor de uma vida humana naquela época?

In [ ]:
# Cria uma tabela cruzada para ver a contagem de sobreviventes por classe
tabela_pclass = pd.crosstab(df['Pclass'], df['Survived'])

# Adiciona os totais para cada linha e coluna
tabela_pclass['Total'] = tabela_pclass.sum(axis=1)

# Calcula a porcentagem de sobreviventes por classe
tabela_pclass['Sobrevivência %'] = round((tabela_pclass[1] / tabela_pclass['Total']) * 100, 2)

# Altera os nomes das colunas para melhor compreensão
tabela_pclass = tabela_pclass.rename(columns={0: 'Não Sobreviveu', 1: 'Sobreviveu'})

print(tabela_pclass)

Análise dos Dados:

A tabela mostra uma correlação inegável entre classe social e sobrevivência.

1ª Classe: 62.96% de sobrevivência.

2ª Classe: 47.28% de sobrevivência.

3ª Classe: Apenas 24.24% de sobrevivência.

Reflexão Filosófica:

Os dados revelam uma hierarquia de valor de vida. O acesso a uma cabine na 1ª classe não era apenas uma questão de conforto; era, literalmente, a diferença entre a vida e a morte. O acesso aos botes salva-vidas, a prioridade no resgate e até a localização das cabines (mais altas no navio) estavam todos correlacionados com a classe social.

O naufrágio do Titanic, que deveria ser um evento de destino cego, foi, na verdade, um reflexo brutal das desigualdades sociais. Os dados, frios e objetivos, confirmam que o valor de uma vida era, em grande parte, determinado pelo status social.

2- Respondendo a segunda Pergunta:

A idade e o gênero determinavam o direito à vida? Qual a relação entre a lógica dos dados e a ética do "mulheres e crianças primeiro"?

In [ ]:
# Cria uma tabela cruzada para ver a contagem de sobreviventes por sexo
tabela_sexo = pd.crosstab(df['Sex'], df['Survived'])

# Adiciona os totais para cada linha e coluna
tabela_sexo['Total'] = tabela_sexo.sum(axis=1)

# Calcula a porcentagem de sobreviventes por sexo
tabela_sexo['Sobrevivência %'] = round((tabela_sexo[1] / tabela_sexo['Total']) * 100, 2)

# Altera os nomes das colunas para melhor compreensão
tabela_sexo = tabela_sexo.rename(columns={0: 'Não Sobreviveu', 1: 'Sobreviveu'})

print(tabela_sexo)

Análise dos Dados:

Mulheres: 74.2% de sobrevivência.

Homens: Apenas 18.89% de sobrevivência.

Reflexão Filosófica:

O dado é brutalmente direto: o gênero era um fator muito mais decisivo para a sobrevivência do que a classe social. A ordem de "mulheres e crianças primeiro" não foi apenas um ideal ético, mas uma política de resgate que se manifestou de forma inegável nos dados.

In [ ]:
# Cria um DataFrame apenas com passageiros do sexo masculino, fazendo uma cópia
homens_df = df[df['Sex'] == 'male'].copy()

In [ ]:
# Cria um DataFrame apenas com passageiros do sexo masculino
homens_df = df[df['Sex'] == 'male']

# Categoriza os homens em crianças (até 16 anos) e adultos
homens_df['Categoria'] = homens_df['Age'].apply(
    lambda age: 'Crianca' if age <= 16 else 'Adulto'
)

# Cria uma tabela cruzada para ver a sobrevivência de homens (crianças vs. adultos)
tabela_criancas_adultos = pd.crosstab(homens_df['Categoria'], homens_df['Survived'])

# Adiciona a porcentagem de sobrevivência
tabela_criancas_adultos['Sobrevivência %'] = round((tabela_criancas_adultos[1] / tabela_criancas_adultos.sum(axis=1)) * 100, 2)

# Altera os nomes das colunas
tabela_criancas_adultos = tabela_criancas_adultos.rename(columns={0: 'Não Sobreviveu', 1: 'Sobreviveu'})

print(tabela_criancas_adultos)

Análise dos Dados:

Homens Adultos: Apenas 16.28% de sobrevivência.

Meninos (Criança): 43.64% de sobrevivência.

Reflexão Filosófica:

Os números falam por si. A política de "mulheres e crianças primeiro" foi de fato aplicada. A sobrevivência dos meninos foi significativamente maior que a dos homens adultos. O que a ética prescrevia, a lógica dos dados confirmou: em uma situação de escassez (de botes salva-vidas), vidas foram priorizadas, e essa prioridade foi baseada na vulnerabilidade percebida

Em suma, a análise mostra que a idade e o gênero foram, de fato, os fatores determinantes para o direito à vida. O que parecia uma ordem ética e humanitária se revela uma regra estatística, com o valor de uma vida sendo quantificado pela sua idade e sexo.

3- Respondendo a Terceira Pergunta:

O que os dados faltantes (missing values) nos dizem sobre o que não sabemos? Isso pode representar a incerteza e o caos de um evento como o naufrágio.

In [ ]:
# Contar o número de valores ausentes por coluna
valores_ausentes_finais = df.isnull().sum()

# Filtrar apenas as colunas com valores ausentes
df_valores_ausentes_finais = valores_ausentes_finais[valores_ausentes_finais > 0]

# Exibir o resultado
print("Valores ausentes após a limpeza:")
print(df_valores_ausentes_finais)

if df_valores_ausentes_finais.empty:
    print("\nO seu DataFrame está 100% limpo!")

In [ ]:
# Descarta a coluna 'Cabin'
df = df.drop('Cabin', axis=1, errors='ignore')

# Verificamos a nova contagem de valores ausentes para confirmar a remoção
print(df.isnull().sum())

Os valores ausentes na coluna Cabin mostra a incerteza e a falta de informação, levando à decisão pragmática de descartar a coluna. Já os poucos valores ausentes em Age e Embarked puderam ser preenchidos de forma inteligente, preservando o máximo de informação

4- Respondendo a quarta pergunta

In [ ]:
from sklearn.model_selection import train_test_split

# Definir a variável independente (características)
# Usar apenas as colunas numéricas ou já tratadas
X = df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]

# Definir a variável dependente (o que queremos prever)
y = df['Survived']

print("Variáveis independentes (X) prontas.")
print("Variável dependente (y) pronta.")

In [ ]:
# Dividir os dados em conjuntos de treino e teste
# O 'random_state' garante que a divisão seja a mesma sempre que o código for executado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Exibir o tamanho dos novos conjuntos
print(f'Tamanho do conjunto de treino (X_train): {X_train.shape[0]}')
print(f'Tamanho do conjunto de teste (X_test): {X_test.shape[0]}')

In [ ]:
# Importar o modelo de Árvore de Decisão
from sklearn.tree import DecisionTreeClassifier

# Criar uma instância do modelo
modelo_arvore = DecisionTreeClassifier(random_state=42)

# Treinar o modelo usando os dados de treino
modelo_arvore.fit(X_train, y_train)

print("Modelo de Árvore de Decisão treinado com sucesso.")

In [ ]:
# Importar a função de métrica de acurácia
from sklearn.metrics import accuracy_score

# Fazer as previsões no conjunto de teste
y_pred = modelo_arvore.predict(X_test)

# Calcular a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)

# Exibir a acurácia
print(f"Acurácia do modelo: {acuracia:.2f}")

Resposta da quarta pergunta:

Sim, uma acurácia de 70% demonstra que a lógica e os padrões extraídos dos dados históricos têm um poder preditivo. O modelo de Machine Learning, que apenas "viu" dados de treino, foi capaz de prever corretamente o destino de 7 em cada 10 novos passageiros.

O modelo não "sabe" que o Titanic afundou, ou que existia uma regra de "mulheres e crianças primeiro." Ele simplesmente aprendeu, a partir dos dados, que ser do sexo feminino, ser uma criança ou ter uma classe mais alta eram características com alta correlação para a sobrevivência. 

A lógica do modelo refletiu a realidade do evento.


A acurácia é uma boa métrica de partida, mas em ciência de dados, ela não conta a história toda. Um modelo com 70% de acurácia pode ser ótimo, mas em alguns casos, pode ser enganador.

5- Respondendo a quinta Pergunta

In [ ]:
# Importar a Matriz de Confusão
from sklearn.metrics import confusion_matrix

# Calcular a matriz de confusão
matriz_confusao = confusion_matrix(y_test, y_pred)

print("Matriz de Confusão:")
print(matriz_confusao)

Analisando cada número com uma lente filosófica e pragmática:

Verdadeiros Negativos (VN = 81): O modelo previu que 81 pessoas não sobreviveriam e elas realmente não sobreviveram. São os acertos do modelo em prever o destino trágico.

Falsos Positivos (FP = 24): O modelo previu que 24 pessoas sobreviveriam, mas elas não sobreviveram. Este é um tipo de erro perigoso. O modelo deu uma falsa esperança.

Falsos Negativos (FN = 30): O modelo previu que 30 pessoas não sobreviveriam, mas elas realmente sobreviveram. Este é outro tipo de erro. O modelo subestimou a capacidade de sobrevivência dessas pessoas.

Verdadeiros Positivos (VP = 44): O modelo previu que 44 pessoas sobreviveriam e elas realmente sobreviveram. São os acertos do modelo em prever o final feliz.

A acurácia de 70% (81 + 44) / (81 + 24 + 30 + 44) = 125/179 ≈ 0.6987 foi confirmada pelos números. No entanto, a matriz de confusão dá uma visão mais detalhada: o modelo é mais preciso em prever a morte (acertou 81 de 111 casos) do que a sobrevivência (acertou 44 de 74 casos).

O modelo, portanto, é bom em prever o destino trágico, mas tem mais dificuldade em prever a sobrevivência. Essa é a incerteza da previsão que queríamos explorar.

6- Respondendo a sexta pergunta

In [ ]:
# Importar o modelo de Random Forest
from sklearn.ensemble import RandomForestClassifier

# Criar uma instância do modelo
modelo_random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
modelo_random_forest.fit(X_train, y_train)

print("Modelo de Random Forest treinado com sucesso.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calcular a matriz de confusão
from sklearn.metrics import confusion_matrix
y_pred_rf = modelo_random_forest.predict(X_test)
matriz_confusao_rf = confusion_matrix(y_test, y_pred_rf)

# Criar visualização
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusao_rf, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Não Sobreviveu', 'Sobreviveu'],
            yticklabels=['Não Sobreviveu', 'Sobreviveu'])

plt.xlabel('Previsão do Modelo')
plt.ylabel('Realidade')
plt.title('Matriz de Confusão do Modelo Random Forest')
plt.show()

A Filosofia do Erro Visual | A Matriz de Confusão Revela:

A matriz de confusão dá uma visão detalhada sobre os acertos e erros do modelo:

Verdadeiros Negativos (VN): 84. O modelo previu que 84 pessoas não sobreviveriam, e elas realmente não sobreviveram. São os acertos do modelo em prever a morte, um número maior do que os 81 acertos do nosso modelo anterior.

Falsos Positivos (FP): 21. O modelo previu que 21 pessoas sobreviveriam, mas elas não sobreviveram. O modelo deu uma "falsa esperança" a essas pessoas, um número menor que os 24 erros do modelo anterior.

Falsos Negativos (FN): 21. O modelo previu que 21 pessoas não sobreviveriam, mas elas realmente sobreviveram. O modelo subestimou a capacidade de sobrevivência dessas pessoas, um número significativamente menor que os 30 erros do modelo anterior.

Verdadeiros Positivos (VP): 53. O modelo previu que 53 pessoas sobreviveriam, e elas realmente sobreviveram. Os acertos do modelo em prever a sobrevivência, um número maior que os 44 acertos do nosso modelo anterior.

O Random Forest teve um desempenho visivelmente superior ao da Árvore de Decisão simples. 

A acurácia, que seria (84+53)/(84+21+21+53) = 137/179, é de aproximadamente 76,5%, o que é uma melhoria significativa em relação aos 70% do modelo anterior.

7- Respondendo a sétima pergunta através das métricas Precisão, Recall, F1-Score e o Relatório de Classificação.

In [ ]:
# Importar as métricas de avaliação
from sklearn.metrics import classification_report, roc_auc_score

# Previsões no conjunto de teste com o modelo Random Forest
y_pred_rf = modelo_random_forest.predict(X_test)

# Gerar o relatório de classificação
relatorio = classification_report(y_test, y_pred_rf)

print("Relatório de Classificação do Random Forest:")
print(relatorio)

A Filosofia da Verdadeira Precisão: O Que as Métricas Revelam

Para a pergunta : "Qual a verdadeira 'precisão' do nosso modelo? Ele é confiável em prever a vida e a morte?"

A resposta, de acordo com o relatório, é uma nuance.

Analises da tabela linha por linha, para as classes 0 (Não Sobreviveu) e 1 (Sobreviveu):

Classe 0 (Não Sobreviveu):

Precisão: 0.80. De todas as vezes que o modelo previu que alguém não sobreviveria, ele acertou 80% das vezes.

Recall: 0.80. De todas as pessoas que realmente não sobreviveram, o modelo conseguiu identificar 80% delas.

F1-Score: 0.80. Um bom equilíbrio entre precisão e recall.

Classe 1 (Sobreviveu):

Precisão: 0.72. De todas as vezes que o modelo previu que alguém sobreviveria, ele acertou 72% das vezes.

Recall: 0.72. De todas as pessoas que realmente sobreviveram, o modelo conseguiu identificar 72% delas.

F1-Score: 0.72. Também um bom equilíbrio, mas a performance é ligeiramente inferior à da classe 0.

Enfim, o modelo é melhor em prever a morte (classe 0) do que a sobrevivência (classe 1).

8- Respondendo a oitava pergunta através da Validação Cruzada (k-fold Cross-Validation),

-- Vou usar o cross_val_score do scikit-learn para fazer essa avaliação

In [ ]:
# Importar a função de validação cruzada
from sklearn.model_selection import cross_val_score

# Avaliar o modelo com 5-fold cross-validation
scores = cross_val_score(modelo_random_forest, X, y, cv=5, scoring='accuracy')

print("Acurácia por fold (5 rodadas):")
print(scores)

print(f"\nAcurácia média: {scores.mean():.2f}")
print(f"Desvio padrão da acurácia: {scores.std():.2f}")

A Filosofia da Consistência: O Que a Validação Cruzada Nos Diz?

Para a pergunta "A previsão é uma coincidência do destino, ou é uma verdade estatística consistente?"

Os números refletem a resposta:

Acurácia Média: 0.70

Desvio Padrão: 0.02

Acurácia Média de 0.70: Isso significa que, em média, o modelo de Random Forest acerta 70% das vezes. 

A acurácia anterior de (0.76) foi um bom resultado, mas essa média de 0.70 é uma estimativa muito mais confiável do desempenho real do modelo.

Desvio Padrão de 0.02: Esse número é ainda mais importante. Um desvio padrão baixo (perto de zero) significa que a performance do modelo é consistente em diferentes subconjuntos de dados. Se o desvio padrão fosse alto, isso indicaria que a acurácia de 76% foi, em parte, sorte. 
Um desvio padrão de 0.02 dá a certeza de que a previsão é uma verdade estatística, e não uma coincidência.

Em resumo, com base em dados, os padrões do passado são consistentes e podem, com um grau de confiabilidade, ser usados para prever o futuro.

9- Respondendo a nona pergunta através da engenharia de features que é o coração da ciência de dados:

Para responder a isso, vou criar duas novas features que combinam informações das colunas existentes:

FamilySize: O tamanho da família, combinando o número de irmãos/cônjuges (SibSp) e pais/crianças (Parch), mais o próprio passageiro.

IsAlone: Uma variável binária que nos diz se o passageiro estava viajando sozinho ou não.

Essas duas colunas, apesar de simples, têm um grande poder preditivo, pois a chance de sobrevivência pode ser muito diferente para quem viaja em família versus quem está sozinho

In [ ]:
# Criar a nova coluna 'FamilySize'
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Exibir as primeiras linhas para verificar
print(df[['SibSp', 'Parch', 'FamilySize']].head())

In [ ]:
# Criar a nova coluna 'IsAlone'
# O valor será 1 se FamilySize for 1 (sozinho) e 0 caso contrário
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# Exibir as primeiras linhas para verificar
print(df[['FamilySize', 'IsAlone']].head())

10- Respondendo a décima pergunta:

In [ ]:
# Definir o novo X com as features adicionais
X_novo = df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone']]

# Treinar e avaliar o modelo com 5-fold cross-validation
scores_novo = cross_val_score(modelo_random_forest, X_novo, y, cv=5, scoring='accuracy')

print("Acurácia por fold (5 rodadas) com as novas features:")
print(scores_novo)

print(f"\nAcurácia média: {scores_novo.mean():.2f}")
print(f"Desvio padrão da acurácia: {scores_novo.std():.2f}")

A Filosofia da União: O que a Nova Análise diz?

A resposta para esta questão, de acordo com as nova atualizações do modelo, é que, neste caso, o novo conhecimento sobre a dinâmica familiar NÂO melhorou a performance geral do modelo.

Anteriormente (sem FamilySize e IsAlone):

Acurácia Média: 0.70

Desvio Padrão: 0.02

Agora (com FamilySize e IsAlone):

Acurácia Média: 0.70

Desvio Padrão: 0.02

Os números são idênticos. Isso não significa que a intuição estava errada, mas sim que o modelo Random Forest, que é um algoritmo poderoso, já era capaz de inferir a informação sobre a dinâmica familiar a partir das colunas SibSp e Parch sozinhas. 

A criação de FamilySize e IsAlone não adicionou um conhecimento novo o suficiente para impactar a performance do modelo.

Isso é uma lição crucial em ciência de dados: nem toda engenharia de features resulta em um ganho de performance. 

O modelo, neste caso, já era "sábio" o suficiente para inferir a informação.